In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#경로 설정 
import os
os.chdir('/content/drive/MyDrive/A2W/제출용 코드/preprocessing/tree_preprocessing/data')

In [ ]:
pip install pyproj

     |████████████████████████████████| 6.6 MB 5.1 MB/s 


In [ ]:
pip install rtree

     |████████████████████████████████| 994 kB 5.1 MB/s 


In [ ]:
pip install geopandas

     |████████████████████████████████| 994 kB 5.0 MB/s 
     |████████████████████████████████| 15.4 MB 37 kB/s 


In [ ]:
# 필수 사용 라이브러리
import pandas as pd # 데이터 프레임화를 위한 라이브러리
import seaborn as sns # Heatmap 사용을 위한 라이브러리
import geopandas as gpd # GIS 데이터가 있는 데이터 사용을 위한 라이브러리
from shapely.geometry import Point, Polygon, LineString # 최대 면적으로 토양도를 매칭하기 위한 라이브러리

In [ ]:
# 경상북도1 임상도 파일 읽기
tree_gyeongbuk1 = gpd.read_file("tree_gyeongbuk1.shp", encoding = 'utf-8')
#경상북도2 임상도 파일 읽기
tree_gyeongbuk2 = gpd.read_file("tree_gyeongbuk2.shp", encoding = 'utf-8')
# 경상남도 임상도 파일 읽기
tree_gyeongnam = gpd.read_file("tree_gyeongnam.shp", encoding = 'utf-8')
tree_gyeongbuk1.info()
tree_gyeongbuk2.info()
tree_gyeongnam.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 433195 entries, 0 to 433194
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   STORUNST    433195 non-null  object  
 1   FROR_CD     433195 non-null  object  
 2   FRTP_CD     433195 non-null  object  
 3   KOFTR_GROU  433195 non-null  object  
 4   DMCLS_CD    372303 non-null  object  
 5   AGCLS_CD    372303 non-null  object  
 6   DNST_CD     372303 non-null  object  
 7   HEIGHT      370566 non-null  object  
 8   LDMARK_STN  433195 non-null  object  
 9   MAP_LABEL   433195 non-null  object  
 10  ETC_PCMTT   33975 non-null   object  
 11  geometry    433195 non-null  geometry
dtypes: geometry(1), object(11)
memory usage: 39.7+ MB
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 355846 entries, 0 to 355845
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype   
---  ------      --------------   -----   
 0   STORUNST    35

In [ ]:
tree_all = pd.concat([tree_gyeongbuk1,tree_gyeongbuk2,tree_gyeongnam], axis =0,join='outer')
tree_all.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1143094 entries, 0 to 354052
Data columns (total 12 columns):
 #   Column      Non-Null Count    Dtype   
---  ------      --------------    -----   
 0   STORUNST    1143094 non-null  object  
 1   FROR_CD     1143094 non-null  object  
 2   FRTP_CD     1143094 non-null  object  
 3   KOFTR_GROU  1143094 non-null  object  
 4   DMCLS_CD    970818 non-null   object  
 5   AGCLS_CD    970818 non-null   object  
 6   DNST_CD     970818 non-null   object  
 7   HEIGHT      968783 non-null   object  
 8   LDMARK_STN  1143094 non-null  object  
 9   MAP_LABEL   1143094 non-null  object  
 10  ETC_PCMTT   104954 non-null   object  
 11  geometry    1143094 non-null  geometry
dtypes: geometry(1), object(11)
memory usage: 113.4+ MB


In [ ]:
tree_all = tree_all.astype({'KOFTR_GROU': 'int'})
tree_all = tree_all[tree_all['KOFTR_GROU']<=80]
tree_all = tree_all.astype({'KOFTR_GROU': 'object'})
tree_all = tree_all.drop(["LDMARK_STN", 'MAP_LABEL', "ETC_PCMTT"], axis=1)
tree_all

,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,DNST_CD,HEIGHT,geometry
186,1,1,1,10,0,1,A,00,"POLYGON ((1185773.031 1787116.515, 1185769.430..."
187,1,1,1,11,0,1,A,00,"POLYGON ((1185704.777 1787453.703, 1185699.711..."
188,1,1,1,11,1,2,A,04,"POLYGON ((1186094.261 1784497.470, 1186098.456..."
189,1,1,1,11,1,2,A,12,"POLYGON ((1180789.172 1779337.255, 1180790.100..."
190,1,1,1,11,1,2,B,08,"POLYGON ((1161762.503 1781881.451, 1161764.808..."
...,...,...,...,...,...,...,...,...,...
354044,1,2,3,77,2,5,C,16,"POLYGON ((1052793.739 1758429.049, 1052794.385..."
354045,1,2,3,77,2,6,C,14,"POLYGON ((1053353.184 1758878.063, 1053357.877..."
354046,1,2,3,77,2,6,C,14,"POLYGON ((1054514.581 1759293.136, 1054519.075..."
354047,1,2,3,77,2,6,C,18,"POLYGON ((1053720.366 1758689.118, 1053716.792..."


In [ ]:
#경로 설정 
import os
os.chdir('/content/drive/MyDrive/A2W/데이터')

In [ ]:
sum_hg_all_g15_sltp_geo = gpd.GeoDataFrame.from_file("sum_hg_all_test_geo.shp", encoding = 'utf-8')
sum_hg_all_g15_sltp_geo

In [ ]:
# Geopandas의 sjoin을 이용하여 임상도 데이터와 (토양도 + 행정동) 데이터 합치기
result = gpd.sjoin(sum_hg_all_g15_sltp_geo, tree_all, how='right', op='intersects')
result["area"] = result.geometry.area
rmp = result.groupby('umd2')['area'].agg(**{'area':'max'}).reset_index()
merge_outer = pd.merge(result,rmp,on=['umd2','area'])
merge_outer

,index_left,umd2,landslide,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,DNST_CD,HEIGHT,geometry,area
0,21.0,포항시 남구 연일읍,1.0,2,26,249.7,29.9,303.5,3,12,01,1,10,03,15.0,1,2,1,15,2,4,C,12,"POLYGON ((1161070.729 1784952.222, 1161072.882...",1.570734e+05
1,111.0,경주시 양북면,1.0,1,11,381.3,32.6,106.0,3,12,06,3,30,02,2.0,1,2,2,34,1,3,C,12,"POLYGON ((1168959.951 1765055.653, 1168958.778...",1.025227e+05
2,12.0,포항시 북구 기계면,1.0,2,26,337.4,36.7,229.3,3,12,01,1,10,03,15.0,1,2,1,11,2,5,C,14,"POLYGON ((1147859.224 1793336.613, 1147862.709...",2.154991e+05
3,13.0,포항시 북구 기북면,1.0,2,26,337.4,36.7,229.3,3,12,01,1,10,03,15.0,1,2,1,11,2,5,C,14,"POLYGON ((1147859.224 1793336.613, 1147862.709...",2.154991e+05
4,11.0,청송군 부남면,0.0,1,11,676.2,30.9,159.8,2,12,01,1,10,03,1.0,1,1,2,44,0,1,A,00,"POLYGON ((1148740.650 1805084.275, 1148741.392...",6.025230e+05
5,19.0,포항시 북구 장량동,1.0,2,23,72.3,23.4,203.4,4,07,01,1,10,03,24.0,1,1,1,15,1,2,B,10,"POLYGON ((1168495.097 1789363.073, 1168490.825...",3.878158e+05
6,24.0,포항시 북구 양학동,1.0,2,23,70.4,22.6,342.7,4,07,01,1,10,03,15.0,1,2,1,15,1,3,C,12,"POLYGON ((1165315.852 1784938.822, 1165320.199...",5.983435e+04
7,25.0,포항시 북구 용흥동,1.0,2,23,70.4,22.6,342.7,4,07,01,1,10,03,15.0,1,2,1,15,1,3,C,12,"POLYGON ((1165315.852 1784938.822, 1165320.199...",5.983435e+04
8,20.0,포항시 북구 환여동,1.0,2,24,52.9,16.2,254.0,4,07,01,1,10,03,15.0,1,2,1,15,2,4,C,12,"POLYGON ((1172652.015 1788709.315, 1172653.575...",2.484205e+05
9,23.0,포항시 북구 중앙동,1.0,2,23,30.8,19.2,39.5,4,07,01,1,10,03,15.0,1,2,2,30,0,1,A,00,"POLYGON ((1168176.870 1785724.212, 1168181.131...",1.002737e+05


In [ ]:
# HEIGHT 변수 확인
result[result['umd2'] == '영천시 완산동']

,index_left,umd2,landslide,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,DNST_CD,HEIGHT,geometry,area
288874,4.0,영천시 완산동,1.0,2,23,125.3,17.3,352.2,3,12,01,1,10,03,15.0,1,1,1,14,1,3,C,None,"POLYGON ((1131091.597 1774145.932, 1131095.207...",4651.288756
288878,4.0,영천시 완산동,1.0,2,23,125.3,17.3,352.2,3,12,01,1,10,03,15.0,1,1,1,14,1,3,C,None,"POLYGON ((1131692.097 1774809.456, 1131697.003...",66084.540771
288898,4.0,영천시 완산동,1.0,2,23,125.3,17.3,352.2,3,12,01,1,10,03,15.0,1,1,1,14,1,4,C,None,"POLYGON ((1131724.917 1774366.441, 1131729.822...",51791.521088
289247,4.0,영천시 완산동,1.0,2,23,125.3,17.3,352.2,3,12,01,1,10,03,15.0,1,1,1,14,3,5,C,None,"POLYGON ((1130974.175 1774513.027, 1130974.864...",31021.374221
289248,4.0,영천시 완산동,1.0,2,23,125.3,17.3,352.2,3,12,01,1,10,03,15.0,1,1,1,14,3,5,C,None,"POLYGON ((1131110.612 1774598.384, 1131117.626...",40196.147152
290831,4.0,영천시 완산동,1.0,2,23,125.3,17.3,352.2,3,12,01,1,10,03,15.0,1,2,1,11,3,5,C,None,"POLYGON ((1131349.968 1774727.371, 1131352.042...",42941.686015
291990,4.0,영천시 완산동,1.0,2,23,125.3,17.3,352.2,3,12,01,1,10,03,15.0,1,2,2,30,1,3,C,None,"POLYGON ((1130853.182 1774077.551, 1130855.967...",30074.985361
291991,4.0,영천시 완산동,1.0,2,23,125.3,17.3,352.2,3,12,01,1,10,03,15.0,1,2,2,30,1,3,C,None,"POLYGON ((1130734.633 1774360.933, 1130738.622...",15124.407868
292518,4.0,영천시 완산동,1.0,2,23,125.3,17.3,352.2,3,12,01,1,10,03,15.0,1,2,2,30,1,4,C,None,"POLYGON ((1131494.406 1774442.016, 1131498.592...",52348.041412
292544,4.0,영천시 완산동,1.0,2,23,125.3,17.3,352.2,3,12,01,1,10,03,15.0,1,2,2,30,2,4,B,None,"POLYGON ((1131068.666 1774493.326, 1131072.161...",41824.521645


In [ ]:
merge_outer.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 287 entries, 0 to 286
Data columns (total 25 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   index_left  287 non-null    float64 
 1   umd2        287 non-null    object  
 2   landslide   287 non-null    float64 
 3   PRRCK_LARG  287 non-null    object  
 4   PRRCK_MDDL  287 non-null    object  
 5   LOCTN_ALTT  287 non-null    float64 
 6   LOCTN_GRDN  287 non-null    float64 
 7   EIGHT_AGL   287 non-null    float64 
 8   CLZN_CD     287 non-null    object  
 9   TPGRP_TPCD  287 non-null    object  
 10  PRDN_FOM_C  287 non-null    object  
 11  SLANT_TYP   287 non-null    object  
 12  SLDPT_TPCD  287 non-null    object  
 13  SCSTX_CD    287 non-null    object  
 14  SLTP_CD     287 non-null    float64 
 15  STORUNST    287 non-null    object  
 16  FROR_CD     287 non-null    object  
 17  FRTP_CD     287 non-null    object  
 18  KOFTR_GROU  287 non-null    object  
 19  

In [ ]:
merge_outer['KOFTR_GROU'].value_counts()

11    119
34     50
30     41
15     23
77     19
32     12
12      7
13      3
14      3
17      2
39      2
33      2
31      1
49      1
44      1
0       1
Name: KOFTR_GROU, dtype: int64

In [ ]:
# 행정동 경계 파일 읽기
hangjung_gubun = gpd.GeoDataFrame.from_file("umd.shp", encoding = 'utf-8')
# 산사태와 합치기 위한 새로운 변수 umd2(ssg+umd) 생성하기
hangjung_gubun_si = pd.read_csv('umd_si.csv', encoding='utf-8')
hangjung_gubun["sgg"] = hangjung_gubun_si
hangjung_gubun["umd2"] = hangjung_gubun["sgg"] + " " + hangjung_gubun["ADM_DR_NM"]
hangjung_gubun = hangjung_gubun.drop([ "BASE_DATE", "ADM_DR_CD", 'sgg', 'ADM_DR_NM'], axis= 1)
hangjung_gubun

,geometry,umd2
0,"MULTIPOLYGON (((1184734.239 1774617.254, 11847...",포항시 남구 구룡포읍
1,"POLYGON ((1160923.832 1786524.018, 1160962.498...",포항시 남구 연일읍
2,"POLYGON ((1172424.168 1778414.742, 1172426.383...",포항시 남구 오천읍
3,"POLYGON ((1167538.928 1778439.770, 1167541.585...",포항시 남구 대송면
4,"MULTIPOLYGON (((1181368.959 1784712.163, 11813...",포항시 남구 동해면
...,...,...
632,"POLYGON ((1056287.745 1722189.578, 1056335.116...",합천군 쌍백면
633,"POLYGON ((1052540.942 1719533.002, 1052573.371...",합천군 삼가면
634,"POLYGON ((1044199.527 1722837.244, 1044218.563...",합천군 가회면
635,"POLYGON ((1046756.129 1730112.827, 1046808.008...",합천군 대병면


In [ ]:
null_region = ['포항시 남구 장기면', '포항시 남구 제철동', '경주시 감포읍', '울릉군 울릉읍', '울릉군 북면']

In [ ]:
df = hangjung_gubun[hangjung_gubun['umd2'].isin(null_region)]
df

,geometry,umd2
5,"POLYGON ((1178077.432 1777146.779, 1178116.217...",포항시 남구 장기면
9,"MULTIPOLYGON (((1171837.211 1781936.253, 11719...",포항시 남구 제철동
29,"MULTIPOLYGON (((1179480.710 1751536.956, 11794...",경주시 감포읍
329,"MULTIPOLYGON (((1387711.920 1924430.817, 13877...",울릉군 울릉읍
331,"MULTIPOLYGON (((1295660.880 1954076.325, 12956...",울릉군 북면


In [ ]:
# Geopandas의 sjoin을 이용하여 임상도 데이터와 결측값 데이터 합치기
result2 = gpd.sjoin(df, tree_all, how='right', op='intersects')
result2["area"] = result2.geometry.area
rmp2 = result2.groupby('umd2')['area'].agg(**{'area':'max'}).reset_index()
merge_outer2 = pd.merge(result2,rmp2,on=['umd2','area'])
merge_outer2

,index_left,umd2,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,DNST_CD,HEIGHT,geometry,area
0,9.0,포항시 남구 제철동,1,2,1,11,2,4,C,10,"POLYGON ((1170518.110 1779224.183, 1170517.611...",6.988158e+04
1,5.0,포항시 남구 장기면,1,2,2,30,2,5,C,18,"POLYGON ((1175902.353 1764835.613, 1175899.256...",6.075229e+05
2,29.0,경주시 감포읍,1,2,1,11,2,4,C,12,"POLYGON ((1177173.759 1755108.652, 1177175.779...",8.907106e+05
3,329.0,울릉군 울릉읍,1,2,2,30,1,5,C,16,"POLYGON ((1298005.304 1952596.988, 1298011.683...",1.819176e+06
4,331.0,울릉군 북면,1,2,2,30,1,5,C,16,"POLYGON ((1298005.304 1952596.988, 1298011.683...",1.819176e+06


In [ ]:
merge_outer.columns

Index(['index_left', 'umd2', 'landslide', 'PRRCK_LARG', 'PRRCK_MDDL',
       'LOCTN_ALTT', 'LOCTN_GRDN', 'EIGHT_AGL', 'CLZN_CD', 'TPGRP_TPCD',
       'PRDN_FOM_C', 'SLANT_TYP', 'SLDPT_TPCD', 'SCSTX_CD', 'SLTP_CD',
       'STORUNST', 'FROR_CD', 'FRTP_CD', 'KOFTR_GROU', 'DMCLS_CD', 'AGCLS_CD',
       'DNST_CD', 'HEIGHT', 'geometry', 'area'],
      dtype='object')

In [ ]:
import numpy as np

merge_outer2[['landslide']] = 1
merge_outer2[['PRRCK_LARG', 'PRRCK_MDDL',
       'LOCTN_ALTT', 'LOCTN_GRDN', 'EIGHT_AGL', 'CLZN_CD', 'TPGRP_TPCD',
       'PRDN_FOM_C', 'SLANT_TYP', 'SLDPT_TPCD', 'SCSTX_CD', 'SLTP_CD']] = np.nan

In [ ]:
merge_outer2

,index_left,umd2,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,DNST_CD,HEIGHT,geometry,area,landslide,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD
0,9.0,포항시 남구 제철동,1,2,1,11,2,4,C,10,"POLYGON ((1170518.110 1779224.183, 1170517.611...",6.988158e+04,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5.0,포항시 남구 장기면,1,2,2,30,2,5,C,18,"POLYGON ((1175902.353 1764835.613, 1175899.256...",6.075229e+05,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,29.0,경주시 감포읍,1,2,1,11,2,4,C,12,"POLYGON ((1177173.759 1755108.652, 1177175.779...",8.907106e+05,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,329.0,울릉군 울릉읍,1,2,2,30,1,5,C,16,"POLYGON ((1298005.304 1952596.988, 1298011.683...",1.819176e+06,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,331.0,울릉군 북면,1,2,2,30,1,5,C,16,"POLYGON ((1298005.304 1952596.988, 1298011.683...",1.819176e+06,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
final = pd.concat([merge_outer,merge_outer2], axis =0,join='outer')
final

,index_left,umd2,landslide,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,EIGHT_AGL,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLANT_TYP,SLDPT_TPCD,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,DNST_CD,HEIGHT,geometry,area
0,21.0,포항시 남구 연일읍,1.0,2,26,249.7,29.9,303.5,3,12,01,1,10,03,15.0,1,2,1,15,2,4,C,12,"POLYGON ((1161070.729 1784952.222, 1161072.882...",1.570734e+05
1,111.0,경주시 양북면,1.0,1,11,381.3,32.6,106.0,3,12,06,3,30,02,2.0,1,2,2,34,1,3,C,12,"POLYGON ((1168959.951 1765055.653, 1168958.778...",1.025227e+05
2,12.0,포항시 북구 기계면,1.0,2,26,337.4,36.7,229.3,3,12,01,1,10,03,15.0,1,2,1,11,2,5,C,14,"POLYGON ((1147859.224 1793336.613, 1147862.709...",2.154991e+05
3,13.0,포항시 북구 기북면,1.0,2,26,337.4,36.7,229.3,3,12,01,1,10,03,15.0,1,2,1,11,2,5,C,14,"POLYGON ((1147859.224 1793336.613, 1147862.709...",2.154991e+05
4,11.0,청송군 부남면,0.0,1,11,676.2,30.9,159.8,2,12,01,1,10,03,1.0,1,1,2,44,0,1,A,00,"POLYGON ((1148740.650 1805084.275, 1148741.392...",6.025230e+05
5,19.0,포항시 북구 장량동,1.0,2,23,72.3,23.4,203.4,4,07,01,1,10,03,24.0,1,1,1,15,1,2,B,10,"POLYGON ((1168495.097 1789363.073, 1168490.825...",3.878158e+05
6,24.0,포항시 북구 양학동,1.0,2,23,70.4,22.6,342.7,4,07,01,1,10,03,15.0,1,2,1,15,1,3,C,12,"POLYGON ((1165315.852 1784938.822, 1165320.199...",5.983435e+04
7,25.0,포항시 북구 용흥동,1.0,2,23,70.4,22.6,342.7,4,07,01,1,10,03,15.0,1,2,1,15,1,3,C,12,"POLYGON ((1165315.852 1784938.822, 1165320.199...",5.983435e+04
8,20.0,포항시 북구 환여동,1.0,2,24,52.9,16.2,254.0,4,07,01,1,10,03,15.0,1,2,1,15,2,4,C,12,"POLYGON ((1172652.015 1788709.315, 1172653.575...",2.484205e+05
9,23.0,포항시 북구 중앙동,1.0,2,23,30.8,19.2,39.5,4,07,01,1,10,03,15.0,1,2,2,30,0,1,A,00,"POLYGON ((1168176.870 1785724.212, 1168181.131...",1.002737e+05


In [ ]:
'포항시 남구 장기면', '포항시 남구 제철동', '경주시 감포읍', '울릉군 울릉읍', '울릉군 북면'

In [ ]:
merge_outer.set_geometry(col='geometry', inplace=True)
merge_outer.to_file("merge_soil_tree.shp",encoding='utf-8')

In [ ]:
final = final.drop([ "index_left" ], axis= 1)
final.set_geometry(col='geometry', inplace=True)
final.to_file("merge_soil_tree.shp",encoding='utf-8')

In [ ]:
final = final.drop([ "geometry", 'area' ], axis= 1)
final.to_csv("merge_soil_tree.csv")